## **Section 1 - Taking & Labelling the Images**

Using a USB Camera and Laptop, 1000 images were taken in batches of 5 or 10 at a time.
Changes in the image frames were made by varying

*   Object Orientation
*   Lighting
*   Object Position
*   Object Distance away from the camera
*   and the objects being in and out of the frame

Images taken were sized 640x320 (2:1) to rereduce memory usage especially while training the model later in this notebook

After images were obtained, they were labeled using [labelImg](https://github.com/tzutalin/labelImg) a graphical python tool for annorating images. 

The object classes used for labelling are:
1.   potatoe_fries
2.   chicken
3.   empty_plate
4.   ketchup_portion
5.   ketchup_bottle

The images were then randomized and split into an 80:20 ratio for training and validation/evaluating the model.
The images and labels/annotation files can be found [HERE](https://drive.google.com/drive/folders/1-PQKmn6JBVB2q4mw21Muee8vS3BoCkYE?usp=sharing)

## **Section 2 - Preparation**


Installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library to be used for model evaluation.

Importing the installed packages

In [3]:
!apt-get update
!python -m pip install --upgrade pip

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Reading package lists... Done                          


In [4]:
!pip install numba
!pip install -q tflite-model-maker
!pip install -q tflite-support
!pip install pycocotools

Import the required packages.

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

## **Section 3 - Training the Model**

### Step 1: Load the dataset

* Images in `train_data` is used to train the object detection model.
* Images in `val_data` is used to evaluate and check if the model can generalize/predict well to new images that it hasn't seen before.

In [6]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/notebooks/train',
    '/notebooks/train',
    ['chicken','ketchup_portion', 'ketchup_bottle', 'potatoe_fries', 'empty_plate']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/notebooks/validate',
    '/notebooks/validate',
    ['chicken','ketchup_portion', 'ketchup_bottle', 'potatoe_fries', 'empty_plate']
)

In [5]:
len(train_data)

800

In [6]:
len(val_data)

200

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

The performance of each EfficientDet-Lite model compared to others is shown in the table below.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


## **Architecture: EfficientDet-Lite0**

In [6]:
spec = model_spec.get('efficientdet_lite0')

2022-04-25 09:51:42.874335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 09:51:42.918447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 09:51:42.918885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 09:51:42.926167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 09:51:42.926649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read f

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 5` which takes 200 steps to go through the 800 images in the training dataset for each epoch.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy.

In [7]:
model = object_detector.create(train_data, model_spec=spec, batch_size=5, train_whole_model=True, epochs=50, validation_data=val_data)

Epoch 1/50


2022-04-25 09:52:37.469516: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


160/160 [==============================] - 94s 356ms/step - det_loss: 1.1409 - cls_loss: 0.7008 - box_loss: 0.0088 - reg_l2_loss: 0.0636 - loss: 1.2045 - learning_rate: 0.0071 - gradient_norm: 2.1051 - val_det_loss: 0.6307 - val_cls_loss: 0.3669 - val_box_loss: 0.0053 - val_reg_l2_loss: 0.0636 - val_loss: 0.6944
Epoch 2/50
160/160 [==============================] - 53s 333ms/step - det_loss: 0.5239 - cls_loss: 0.3262 - box_loss: 0.0040 - reg_l2_loss: 0.0637 - loss: 0.5875 - learning_rate: 0.0062 - gradient_norm: 2.4447 - val_det_loss: 0.3871 - val_cls_loss: 0.2212 - val_box_loss: 0.0033 - val_reg_l2_loss: 0.0637 - val_loss: 0.4508
Epoch 3/50
160/160 [==============================] - 53s 332ms/step - det_loss: 0.3902 - cls_loss: 0.2531 - box_loss: 0.0027 - reg_l2_loss: 0.0637 - loss: 0.4539 - learning_rate: 0.0062 - gradient_norm: 2.3691 - val_det_loss: 0.2933 - val_cls_loss: 0.1780 - val_box_loss: 0.0023 - val_reg_l2_loss: 0.0637 - val_loss: 0.3571
Epoch 4/50
160/160 [================

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, the 200 images in the validation dataset to is used evaluate how the model performs against new data it has never seen before.

In [8]:
model.evaluate(val_data)

4/4 [==============================] - 22s 2s/step



{'AP': 0.9132361,
 'AP50': 0.99986994,
 'AP75': 0.995243,
 'APs': 0.6107826,
 'APm': 0.9064384,
 'APl': 0.92719424,
 'ARmax1': 0.6146744,
 'ARmax10': 0.94974965,
 'ARmax100': 0.9512094,
 'ARs': 0.825,
 'ARm': 0.94476503,
 'ARl': 0.957675,
 'AP_/chicken': 0.90475774,
 'AP_/ketchup_portion': 0.89451325,
 'AP_/ketchup_bottle': 0.9064585,
 'AP_/potatoe_fries': 0.9166174,
 'AP_/empty_plate': 0.94383335}

### Step 5: Export Model as a TensorFlow Lite model.

In [9]:
path1 = '/notebooks/models/model0.tflite' # batch = 5; epochs = 50 architecture = efficientdet_lite0

In [10]:
model.export(export_dir='.', tflite_filename=path1)

2022-04-25 10:39:00.912659: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-04-25 10:39:24.373692: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-04-25 10:39:37.104400: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-04-25 10:39:37.104475: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-25 10:39:37.104487: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-04-25 10:39:37.106130: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpfvqnx04q
2022-04-25 10:39:37.252965: I tensorflow/cc/saved_model/reader.cc:107] Reading m

### Step 6:  Evaluate the TensorFlow Lite model.

In [11]:
model.evaluate_tflite(path1, val_data)

200/200 [==============================] - 40s 196ms/step



{'AP': 0.89553547,
 'AP50': 0.99998766,
 'AP75': 0.9938825,
 'APs': 0.56905943,
 'APm': 0.8909096,
 'APl': 0.9037005,
 'ARmax1': 0.6111694,
 'ARmax10': 0.9170122,
 'ARmax100': 0.9170122,
 'ARs': 0.575,
 'ARm': 0.91457975,
 'ARl': 0.92327774,
 'AP_/chicken': 0.89246935,
 'AP_/ketchup_portion': 0.8792171,
 'AP_/ketchup_bottle': 0.8918523,
 'AP_/potatoe_fries': 0.8936617,
 'AP_/empty_plate': 0.92047703}

## **Architecture: EfficientDet-Lite1**

In [9]:
spec =  model_spec.get('efficientdet_lite1')

In [10]:
model = object_detector.create(train_data, model_spec=spec, batch_size=5, train_whole_model=True, epochs=50, validation_data=val_data)

Epoch 1/50


2022-04-25 13:01:04.537441: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


160/160 [==============================] - 144s 612ms/step - det_loss: 1.0585 - cls_loss: 0.6990 - box_loss: 0.0072 - reg_l2_loss: 0.0712 - loss: 1.1297 - learning_rate: 0.0071 - gradient_norm: 2.1851 - val_det_loss: 0.4539 - val_cls_loss: 0.2899 - val_box_loss: 0.0033 - val_reg_l2_loss: 0.0712 - val_loss: 0.5251
Epoch 2/50
160/160 [==============================] - 93s 580ms/step - det_loss: 0.4365 - cls_loss: 0.2940 - box_loss: 0.0028 - reg_l2_loss: 0.0713 - loss: 0.5077 - learning_rate: 0.0062 - gradient_norm: 2.4046 - val_det_loss: 0.3226 - val_cls_loss: 0.2070 - val_box_loss: 0.0023 - val_reg_l2_loss: 0.0713 - val_loss: 0.3938
Epoch 3/50
160/160 [==============================] - 93s 580ms/step - det_loss: 0.3156 - cls_loss: 0.2138 - box_loss: 0.0020 - reg_l2_loss: 0.0713 - loss: 0.3869 - learning_rate: 0.0062 - gradient_norm: 2.1099 - val_det_loss: 0.2124 - val_cls_loss: 0.1426 - val_box_loss: 0.0014 - val_reg_l2_loss: 0.0713 - val_loss: 0.2837
Epoch 4/50
160/160 [===============

In [11]:
model.evaluate(val_data)

4/4 [==============================] - 27s 3s/step



{'AP': 0.94028926,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': 0.8293918,
 'APm': 0.925472,
 'APl': 0.96077496,
 'ARmax1': 0.6271326,
 'ARmax10': 0.97260505,
 'ARmax100': 0.97417724,
 'ARs': 0.9,
 'ARm': 0.9663518,
 'ARl': 0.9845165,
 'AP_/chicken': 0.9167776,
 'AP_/ketchup_portion': 0.9418203,
 'AP_/ketchup_bottle': 0.9336795,
 'AP_/potatoe_fries': 0.94361615,
 'AP_/empty_plate': 0.9655528}

In [12]:
path2 = '/notebooks/models/model1.tflite' # batch = 5; epochs = 50 architecture = efficientdet_lite1

In [13]:
model.export(export_dir='.', tflite_filename=path2)

2022-04-25 14:21:33.708373: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-04-25 14:22:09.473805: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-04-25 14:22:26.354023: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-04-25 14:22:26.354089: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-25 14:22:26.354112: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-04-25 14:22:26.355355: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpyagxwbif
2022-04-25 14:22:26.544494: I tensorflow/cc/saved_model/reader.cc:107] Reading m

In [14]:
model.evaluate_tflite(path2, val_data)

200/200 [==============================] - 90s 447ms/step



{'AP': 0.9185509,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': 0.8089109,
 'APm': 0.90745217,
 'APl': 0.9312441,
 'ARmax1': 0.62046695,
 'ARmax10': 0.9409776,
 'ARmax100': 0.9409776,
 'ARs': 0.825,
 'ARm': 0.9311237,
 'ARl': 0.95132,
 'AP_/chicken': 0.903558,
 'AP_/ketchup_portion': 0.9139276,
 'AP_/ketchup_bottle': 0.90554583,
 'AP_/potatoe_fries': 0.930039,
 'AP_/empty_plate': 0.9396842}

## **Architecture: EfficientDet-Lite2**

In [7]:
spec =  model_spec.get('efficientdet_lite2')

2022-04-25 15:19:42.784970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 15:19:42.858069: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 15:19:42.858599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 15:19:42.861524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 15:19:42.862012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read f

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=5, train_whole_model=True, epochs=50, validation_data=val_data)

Epoch 1/50


In [ ]:
model.evaluate(val_data)

In [ ]:
path3 = '/notebooks/models/model2.tflite' # batch = 5; epochs = 50 architecture = efficientdet_lite2

In [ ]:
model.export(export_dir='.', tflite_filename=path3)

In [ ]:
model.evaluate_tflite(path3, val_data)

## **Architecture: EfficientDet-Lite3**

In [ ]:
spec =  model_spec.get('efficientdet_lite3')

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=5, train_whole_model=True, epochs=50, validation_data=val_data)

In [ ]:
model.evaluate(val_data)

In [ ]:
path4 = '/notebooks/models/model3.tflite' # batch = 5; epochs = 50 architecture = efficientdet_lite3

In [ ]:
model.export(export_dir='.', tflite_filename=path4)

In [ ]:
model.evaluate_tflite(path4, val_data)

## **Architecture: EfficientDet-Lite4**

In [ ]:
spec =  model_spec.get('efficientdet_lite4')

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=5, train_whole_model=True, epochs=50, validation_data=val_data)

In [ ]:
model.evaluate(val_data)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
path5 = '/notebooks/models/model4.tflite' # batch = 5; epochs = 50 architecture = efficientdet_lite4

In [ ]:
model.export(export_dir='.', tflite_filename=path5)

In [ ]:
model.evaluate_tflite(path5, val_data)